In [1]:
import json
import binascii
import os
import urllib3
import certifi
import requests
from datetime import date, datetime, timedelta
import time

In [2]:
#----------------------------------------------------------------------------
# https://julien.danjou.info/handling-multipart-form-data-python/
#----------------------------------------------------------------------------
def encode_multipart_formdata(fields):

    boundary = binascii.hexlify(os.urandom(16)).decode('ascii')
    body = (
        "".join("--%s\r\n"
                "Content-Disposition: form-data; name=\"%s\"\r\n"
                "\r\n"
                "%s\r\n" % (boundary, field, value)
                for field, value in fields.items()) +
        "--%s--\r\n" % boundary
    )
    content_type = "multipart/form-data; boundary=%s" % boundary
    return body, content_type

In [3]:
def get_search_url(search_url_file = 'search_url_list.txt'):

    search_url_list = []

    with open(search_url_file, 'r') as f:
        urls = f.readlines()

    for current_string in urls:
        if current_string[0:1] == '#' or current_string[0:1] == '\n':
            continue
        search_url_list.append(current_string.rstrip())

    return search_url_list


In [4]:
def get_vacancies(url_list, vacancies, log_file_name='vacancy_log.txt'):

    date_from = (date.today() - timedelta(hours=24)).isoformat()
    searches = []
    new_vacancies = []

    with open(log_file_name, 'a') as logfile:

        for url in url_list:
            for page in range(20):
                try:
                    searches.append(requests.get(url + '&page={}&per_page=100&date_from={}'.format(page, date_from)).json())
                except Exception as ex_err:
                    logfile.write('==> ERROR: {} W= Read Vacansies ===> {}'.format(date.today().isoformat(), ex_err) + '\n')

        try:
            for items in searches:
                for j in items['items']:
                    if j['id'] not in vacancies:
                        vacancies.append(j['id'])
                        new_vacancies.append(j['id'])
        except:
            pass

    return new_vacancies

In [5]:
def respond_to_vacancies(vacancies, resume_id, message='', log_file_name='vacancy_log.txt'):

    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

    with open(log_file_name, 'a') as logfile:

        for vacancy_id in vacancies:
            try:

                body, content_type = encode_multipart_formdata({'vacancy_id': '{}'.format(vacancy_id),
                                     'resume_id' : '{}'.format(resume_id),
                                     'message'   : '{}'.format(message)})

                r = http.request('POST', 'https://api.hh.ru/negotiations',
                                        headers = {'Authorization' : 'Bearer {}'.format(user_token_params['access_token']),
                                                  'Accept'        : '*/*',
                                                  'User-Agent'    : 'CIO_jbSearch (ssv.ruby@gmail.com)',
                                                  'Content-Type'  : '{}'.format(content_type)},
                                        body =  '{}'.format(body)
                                      )

                result = 'Vacancy_id: {}, Date: {}, Status: {}, Reason: {}, Response_link: {}'.format(vacancy_id, r.getheader('Date'),
                                                                                r.status, r.reason,
                                                                                'https://hh.ru/applicant{}'.format(r.getheader('Location')))
                logfile.write('{} == Respond Vacansies ===> {}'.format(date.today().isoformat(), result) + '\n')

            except Exception as ex_err:
                logfile.write('{} W= Respond ERROR ===> {}'.format(date.today().isoformat(), ex_err) + '\n')

In [6]:
vacancies = []                                       #  
resume_id = '6e1019f2ff040e03530039888f4b59356a4b71' # 
period    = 900                                      # 

In [7]:
# Get authorisation params
with open('user_token_params.json', 'r', encoding='utf-8') as f:
    user_token_params = json.load(f)

In [8]:
# Get search requests list  (`get_search_url()`)
url_list = get_search_url()

In [ ]:
while True:
    try:
        new_vacancies = get_vacancies(url_list, vacancies)
        respond_to_vacancies(new_vacancies, resume_id)
        time.sleep(period)
    except:
        pass